<a href="https://colab.research.google.com/github/Bisrath1/amharic-ecommerce-ner-extractor/blob/main/Model_Interpretability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch shap lime pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 132.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.3 MB/s eta 0:00:00
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sh

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

# Load the fine-tuned model and tokenizer
model_name = "path_to_your_fine_tuned_model"  # Replace with your model path (e.g., saved XLM-RoBERTa)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Define label mappings (from Task 2)
label_list = ['O', 'B-Product', 'I-Product', 'B-LOC', 'I-LOC', 'B-PRICE', 'I-PRICE']
id2label = {i: label for i, label in enumerate(label_list)}

In [ ]:
import pandas as pd

# Load the CoNLL dataset
def load_conll_data(file_path):
    sentences, labels = [], []
    current_sentence, current_labels = [], []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip() == "":
                if current_sentence:
                    sentences.append(current_sentence)
                    labels.append(current_labels)
                    current_sentence, current_labels = [], []
            else:
                token, label = line.strip().split()
                current_sentence.append(token)
                current_labels.append(label)
        if current_sentence:  # Append the last sentence
            sentences.append(current_sentence)
            labels.append(current_labels)
    return sentences, labels

# Load your CoNLL dataset
file_path = "path_to_your_conll_file.txt"  # Replace with your CoNLL file path
sentences, true_labels = load_conll_data(file_path)

# Select a subset of 5–10 sentences for interpretability
sample_sentences = sentences[:10]
sample_labels = true_labels[:10]

In [ ]:
import shap
import numpy as np

# Define a prediction function for SHAP
def predict_for_shap(inputs):
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits.cpu().numpy()  # Shape: (batch_size, seq_len, num_labels)
    return logits

# Prepare a single sentence for SHAP
sample_sentence = sample_sentences[0]  # Example: First sentence
inputs = tokenizer(sample_sentence, is_split_into_words=True, return_tensors="pt", padding=True, truncation=True)

# Create a SHAP explainer
explainer = shap.DeepExplainer(predict_for_shap, inputs)

# Compute SHAP values
shap_values = explainer.shap_values(inputs)

# Visualize SHAP values for a specific label (e.g., B-Product)
label_idx = label_list.index("B-Product")
shap.summary_plot(shap_values[label_idx], feature_names=tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]))

In [ ]:
# Predict labels for a sample sentence
def predict_labels(sentence):
    inputs = tokenizer(sentence, is_split_into_words=True, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)[0].cpu().numpy()
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    predicted_labels = [id2label[pred] for pred in predictions]
    return tokens, predicted_labels

# Analyze a difficult case
difficult_sentence = sample_sentences[2]  # Example: Choose a sentence with potential ambiguity
tokens, predicted_labels = predict_labels(difficult_sentence)
true_labels_subset = sample_labels[2]

# Print comparison
print("Token\tTrue Label\tPredicted Label")
for token, true_label, pred_label in zip(tokens, true_labels_subset, predicted_labels):
    print(f"{token}\t{true_label}\t{pred_label}")

# Use SHAP to diagnose errors
inputs = tokenizer(difficult_sentence, is_split_into_words=True, return_tensors="pt", padding=True, truncation=True)
shap_values = explainer.shap_values(inputs)
shap.summary_plot(shap_values[label_list.index("B-Product")], feature_names=tokens)